# Импорт библиотек

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

import os

import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from transformers import pipeline
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

from datasets import Dataset

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import *

import joblib
import cloudpickle
import pickle

import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

from nltk.corpus import stopwords
from functools import lru_cache

from pymorphy3 import MorphAnalyzer
import re
import emoji
import torch

# Настройка конфигураций

In [ ]:
from omegaconf import OmegaConf
import os

def load_config(config_name):
    """Загрузка конфигурации для конкретного эксперимента"""
    config_path = f"configs/{config_name}.yml"
    
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Конфигурационный файл {config_path} не найден")
    
    # Загружаем основной конфиг
    cfg = OmegaConf.load(config_path)
    
    # Переопределяем переменными окружения
    if 'MLFLOW_TRACKING_URI' in os.environ:
        cfg.mlflow.tracking_uri = os.environ['MLFLOW_TRACKING_URI']
    
    return cfg

# Эксперименты с датасетами

## Первый эксперимент (Лемматизация, удаление знаков пунктуации)

In [ ]:
#!   mlflow server --host 127.0.0.1 --port 8080

In [ ]:
cfg = load_config("base")  # или другое имя вашего конфига

# Теперь можно использовать
mlflow.set_tracking_uri(cfg.mlflow.tracking_uri)

print(f"Tracking URI: {cfg.mlflow.tracking_uri}")

In [ ]:
# Загружаем конфигурацию
cfg = load_config("preprocess_first")

# Инициализация компонентов
analyzer = MorphAnalyzer(lang='ru')

# Получаем стоп-слова из конфигурации
stop_words = nltk.corpus.stopwords.words('russian')
stop_words_cleaned = [
    w for w in stop_words
    if w not in cfg.preprocess.keep_words  # Теперь это должно работать
]

@lru_cache(maxsize=cfg.preprocess.lru_cache_size)
def lemmatization(text):
    return analyzer.parse(text)[0].normal_form

def preprocess_text(text):
    text = re.sub(cfg.preprocess.regex.remove_newlines, " ", text)

    # 2. Фиксируем кривые слова nчипсы → чипсы
    text = re.sub(cfg.preprocess.regex.fix_mistyped_n, r"\1", text)

    # 3. Убираем символы
    text = re.sub(cfg.preprocess.regex.remove_symbols, "", text)

    # 4. Чистим двойные пробелы
    text = re.sub(cfg.preprocess.regex.collapse_spaces, " ", text).strip()

    # 5. Токенизация
    tokens = nltk.word_tokenize(text)
    result = []

    for token in tokens:
        if token not in stop_words_cleaned:
            result.append(lemmatization(token))

    return " ".join(result)

# Проверка работы
test_text = "Всем привет! Какое же неприятное место, нет?\n"
processed = preprocess_text(test_text)
print(f"Исходный: {test_text}")
print(f"Обработанный: {processed}")

In [ ]:
with mlflow.start_run(run_name="First dataset"):

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)

    # Загружаем все файлы из конфига
    annotation_dfs = [
        pd.read_json(path) for path in cfg.preprocess.input_files
    ]

    df_annotations = pd.concat(annotation_dfs)
    
    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]

    # Применяем препроцессинг
    df['span'] = df['span'].apply(preprocess_text)

    display(df)

    # Сохраняем датасет
    df.to_csv(cfg.preprocess.output.dataset_csv, index=False)

    mlflow.log_artifact(cfg.preprocess.output.dataset_csv, "datasets")

    # Сохраняем препроцесс-функцию
    with open(cfg.preprocess.output.preprocess_pickle, "wb") as f:
        cloudpickle.dump(preprocess_text, f)

    mlflow.log_artifact(cfg.preprocess.output.preprocess_pickle, "functions")
    

## Второй эксперимент (Простая очистка от знаков препинания и изменение эмодзи)

In [ ]:
from functools import partial

cfg = load_config('preprocess_second')

def clean_text_only(text, cfg=None):
    """
    Очистка текста с поддержкой конфигурации
    
    Args:
        text: Входной текст
        cfg: Конфигурация (опционально)
    """
    if not isinstance(text, str):
        return ""
    
    # Если конфигурация не передана, используем значения по умолчанию
    lowercase = getattr(cfg, 'clean_only', {}).get('lowercase', True) if cfg else True
    replace_emoji = getattr(cfg, 'clean_only', {}).get('replace_emoji', True) if cfg else True
    remove_punctuation = getattr(cfg, 'clean_only', {}).get('remove_punctuation', True) if cfg else True
    remove_special_chars = getattr(cfg, 'clean_only', {}).get('remove_special_chars', True) if cfg else True
    collapse_spaces = getattr(cfg, 'clean_only', {}).get('collapse_spaces', True) if cfg else True
    
    if lowercase:
        text = text.lower()
    
    # Заменяем эмодзи
    if replace_emoji:
        text = emoji.demojize(text, delimiters=(" ", " "))
    
    # Удаляем переносы и спецсимволы
    if remove_special_chars:
        text = re.sub(r'[\n\r\t]', ' ', text)
    
    # Удаляем пунктуацию
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', ' ', text)
    
    # Убираем лишние пробелы
    if collapse_spaces:
        text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Версия для обратной совместимости (без конфигурации)
def clean_text_only_legacy(text):
    """Легаси версия для обратной совместимости"""
    return clean_text_only(text)


with mlflow.start_run(run_name='Second dataset'):
    
    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)

    # Загружаем все файлы из конфига
    annotation_dfs = [
        pd.read_json(path) for path in cfg.preprocess.input_files
    ]

    df_annotations = pd.concat(annotation_dfs)
    
    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]

    # ИСПРАВЛЕНИЕ: передаем cfg в функцию препроцессинга
    # Вариант 1: Используем lambda
    df['span'] = df['span'].apply(lambda x: clean_text_only(x, cfg))
    
    # Или Вариант 2: Создаем частично примененную функцию
    # clean_text_with_config = partial(clean_text_only, cfg=cfg)
    # df['span'] = df['span'].apply(clean_text_with_config)

    display(df)

    # Сохраняем датасет
    df.to_csv(cfg.preprocess.output.dataset_csv, index=False)

    mlflow.log_artifact(cfg.preprocess.output.dataset_csv, "datasets")

    # ИСПРАВЛЕНИЕ: сохраняем функцию с привязанным конфигом
    # Создаем частично примененную функцию для сохранения
    clean_text_with_config = partial(clean_text_only, cfg=cfg)
    with open(cfg.preprocess.output.preprocess_pickle, "wb") as f:
        cloudpickle.dump(clean_text_with_config, f)

    mlflow.log_artifact(cfg.preprocess.output.preprocess_pickle, "functions")

## Третий эксперимент (Аугментация данных для минорных классов)

In [ ]:
with mlflow.start_run(run_name="Third dataset"):

    cfg = load_config("preprocess_third")

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)


    annotation_dfs = [pd.read_json(path) for path in cfg.preprocess.input_files]
    df_annotations = pd.concat(annotation_dfs, ignore_index=True)


    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]


    class_counts = df['label'].value_counts()
    minority_classes = class_counts[class_counts < class_counts.mean()].index.tolist()

    print(f"Minority classes: {minority_classes}")


    import random

    synonyms = cfg.preprocess.augmentation.synonyms
    n_variants = cfg.preprocess.augmentation.n_variants

    def simple_augmentation(text):
        augmented = []
        for _ in range(n_variants):
            words = text.split()
            new_words = []
            for w in words:
                wl = w.lower()
                if wl in synonyms and random.random() > 0.7:
                    new_words.append(random.choice(synonyms[wl]))
                else:
                    new_words.append(w)
            augmented.append(" ".join(new_words))
        return augmented


    augmented_data = []
    for label in minority_classes:
        samples = df[df['label'] == label]
        for _, row in samples.iterrows():
            for aug_text in simple_augmentation(row["span"]):
                augmented_data.append({"span": aug_text, "label": label})

    df_augmented = pd.DataFrame(augmented_data)


    df_extended = pd.concat([df, df_augmented], ignore_index=True)


    output_path = cfg.preprocess.output.dataset_csv
    df_extended.to_csv(output_path, index=False)

    mlflow.log_artifact(output_path, "datasets")

    print(f"Сохранен датасет с аугментацией: {output_path}")


# Эксперементирование с моделями

## Первый эксперимент (Простая модель)

Будем обучать простую модель: "Наивный Байесовский Классификатор" из `Sklearn`, будем проводить тест на 3 версиях датасета и поймем, какая модель лучше себя покажет при работе с тем или иным датасетом

### Первый датасет

In [ ]:
cfg = load_config("naive_bayes_first")

with mlflow.start_run(run_name='first_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    # Используем пути из конфига
    full_dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
    
    try:
        dataframe_path = client.download_artifacts(first_dataset_latest_run_id, full_dataset_path)
        df = pd.read_csv(dataframe_path)
        print(f"✅ Успешно загружен: {full_dataset_path}")
    except Exception as e:
        print(f"❌ Не удалось загрузить {full_dataset_path}: {e}")

        try:
            dataframe_path = client.download_artifacts(first_dataset_latest_run_id, cfg.data.dataset_file)
            df = pd.read_csv(dataframe_path)
            print(f"✅ Успешно загружен: {cfg.data.dataset_file}")
        except:
            raise ValueError(f"Не удалось загрузить датасет: {full_dataset_path}")
    
    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================


    mlflow.sklearn.autolog(disable=True)
    
    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    # Сохраняем артефакты
    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    # Логируем кастомные метрики
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    # Логируем параметры из конфига
    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Модель успешно обучена и залогирована!")

#### Проверка результатов

In [ ]:
# Загружаем конфиг для инференса
cfg_inference = load_config("inference_bayes_first")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}" 
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")

    try:
        vectorizer_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.vectorizer)
        bayes_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.bayes)
        encoder_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.encoder)
        print("✅ Модели загружены (fallback)")
    except:
        raise ValueError("Не удалось загрузить модели")

# ===========================================================================================
# ПОЛУЧЕНИЕ ФУНКЦИИ ПРЕПРОЦЕССИНГА
# ===========================================================================================


latest_run_dataset = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.preprocess.run_name}"',
    order_by=['attributes.end_time desc']
)

latest_run_dataset_id = latest_run_dataset[0].info.run_id

try:
    art_loc = client.download_artifacts(latest_run_dataset_id, cfg_inference.preprocess.artifact_path)
    print("✅ Функция препроцессинга загружена")
except Exception as e:
    print(f"❌ Ошибка загрузки функции: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================


vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)


with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

# Используем текст из конфига
text = cfg_inference.test_text

preprocessed_text = preprocess_func(text)
done_text = vectorizer.transform([preprocessed_text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')

### Второй датасет

In [ ]:

cfg = load_config("naive_bayes_second")

with mlflow.start_run(run_name='second_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id


    try:
        dataframe_path = client.download_artifacts(first_dataset_latest_run_id, "datasets/second_experiment_dataset.csv")
        df = pd.read_csv(dataframe_path)
        print("✅ Датасет загружен напрямую")
    except:
        try:
            files = client.list_artifacts(first_dataset_latest_run_id, cfg.data.dataset_path)
            dataframe = files[0].path
            dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)
            df = pd.read_csv(dataframe_path)
            print("✅ Датасет загружен через list_artifacts")
        except:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog(disable=True)

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Вторая модель успешно обучена и залогирована!")

#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_bayes_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"', 
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ПОЛУЧЕНИЕ ФУНКЦИИ ПРЕПРОЦЕССИНГА
# ===========================================================================================


latest_run_dataset = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.preprocess.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_dataset:
    raise ValueError(f"Не найден run: {cfg_inference.preprocess.run_name}")

latest_run_dataset_id = latest_run_dataset[0].info.run_id

print("Загружаем функцию препроцессинга...")
try:
    art_loc = client.download_artifacts(latest_run_dataset_id, cfg_inference.preprocess.artifact_path)
    print("✅ Функция препроцессинга загружена")
except Exception as e:
    print(f"❌ Ошибка загрузки функции: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

print("Загружаем функцию препроцессинга...")
with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

# Используем текст из конфига
text = cfg_inference.test_text

print("Выполняем предсказание...")
preprocessed_text = preprocess_func(text)
done_text = vectorizer.transform([preprocessed_text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')

### Третий датасет

In [ ]:
cfg = load_config("naive_bayes_third")

with mlflow.start_run(run_name='third_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")


    try:
        dataframe_path = client.download_artifacts(dataset_run_id, f"datasets/{cfg.data.dataset_file}")
        df = pd.read_csv(dataframe_path)
        print("✅ Датасет загружен напрямую")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/datasets/{cfg.data.dataset_file}",
            "third_experiment_dataset.csv"
        ]
        
        for path in alternative_paths:
            try:
                dataframe_path = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(dataframe_path)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog(disable=True)

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Третья модель успешно обучена и залогирована!")

#### Проверка результатоы

In [ ]:

cfg_inference = load_config("inference_bayes_third")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise


# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

# Используем текст из конфига
text = cfg_inference.test_text

print("Выполняем предсказание...")
done_text = vectorizer.transform([text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')
print('=' * 100)

### Итог

In [ ]:
results = pd.DataFrame({
    'Версия': ['Версия 1', 'Версия 2', 'Версия 3'],
    'F1-Score': [0.363707, 0.344306, 0.344139],
    'Accuracy': [0.413043, 0.391304, 0.391304],
    'Описание': ['Лемматизация, удаление знаков пунктуации', 'Очистка от знаков препинания и emoji to text', 'Никакой очистки']
})

results

## Второй эксперимент (Keras)

### Первый датасет

In [ ]:

cfg = load_config("neural_network_first")

with mlflow.start_run(run_name='first_experiment_neural_network'):
    
    mlflow.set_tag('LSTM', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataframe_path = client.download_artifacts(dataset_run_id, f"{cfg.data.dataset_path}/{cfg.data.dataset_file}")
        df = pd.read_csv(dataframe_path)
        print("✅ Датасет загружен напрямую")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        try:
            files = client.list_artifacts(dataset_run_id, cfg.data.dataset_path)
            dataframe = files[0].path
            dataframe_path = client.download_artifacts(dataset_run_id, dataframe)
            df = pd.read_csv(dataframe_path)
            print("✅ Датасет загружен через list_artifacts")
        except Exception as e2:
            print(f"❌ Ошибка загрузки через list_artifacts: {e2}")
            alternative_paths = [
                cfg.data.dataset_file,
                f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
                "First_version.csv"
            ]
            
            for path in alternative_paths:
                try:
                    dataframe_path = client.download_artifacts(dataset_run_id, path)
                    df = pd.read_csv(dataframe_path)
                    print(f"✅ Датасет загружен: {path}")
                    break
                except:
                    continue
            else:
                raise ValueError("Не удалось загрузить датасет")

    display(df)
    
    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()

    tokenizer = Tokenizer(
        num_words=cfg.tokenizer.num_words,
        oov_token=cfg.tokenizer.oov_token,
        filters=cfg.tokenizer.filters,
        lower=cfg.tokenizer.lower,
        split=cfg.tokenizer.split,
        char_level=cfg.tokenizer.char_level
    )

    tokenizer.fit_on_texts(df['span'])

    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['label']
    )

    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])

    max_len_text = 0
    for i in df['span']:
        max_len_text = max(max_len_text, len(i.split(' ')))
    print(f"Максимальная длина текста: {max_len_text}")


    X_train_pad = pad_sequences(X_train_vec, max_len_text, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, max_len_text, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, max_len_text, padding='post', truncating='post')

    # =====================================================================================================================================
    #                                              ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================

    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])

    # =====================================================================================================================================
    #                                              СОЗДАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    model = Sequential()

    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim, 
        input_length=max_len_text
    ))

    model.add(Bidirectional(LSTM(
        cfg.model.lstm_units, 
        dropout=0.2, 
        recurrent_dropout=0.3
    )))

    model.add(Dropout(cfg.model.dropout_rate))

    model.add(Dense(cfg.model.dense_units, activation='softmax'))

    model.compile(
        optimizer='adam', 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )

    model.fit(
        X_train_pad, 
        y_train, 
        batch_size=cfg.training.batch_size, 
        epochs=cfg.training.epochs, 
        validation_data=(X_val_pad, y_val)
    )

    test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    model.summary()

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred_proba = model.predict(X_test_pad)
    pred_class = np.argmax(pred_proba, axis=1)
    
    f1 = f1_score(pred_class, y_test, average='weighted')
    accuracy = accuracy_score(pred_class, y_test)

    print(f'f1-score у модели равен {f1:.4f}')
    print(f'accuracy у модели равен {accuracy:.4f}')

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('test_loss', test_loss)

    model.save("LSTM_ver_1.keras")

    with open(cfg.model.artifacts.tokenizer, 'wb') as f:
        cloudpickle.dump(tokenizer, f)
    
    with open(cfg.model.artifacts.encoder, 'wb') as f:
        cloudpickle.dump(encoder, f)

    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'models') 
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')

    mlflow.log_params({
        'num_words': cfg.tokenizer.num_words,
        'embedding_dim': cfg.model.embedding_dim,
        'lstm_units': cfg.model.lstm_units,
        'dropout_rate': cfg.model.dropout_rate,
        'batch_size': cfg.training.batch_size,
        'epochs': cfg.training.epochs
    })

    print("✅ Нейронная сеть успешно обучена и залогирована!")

#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.tokenizer}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)
encoder = joblib.load(encoder_file)
tokenizer = joblib.load(tokenizer_file)

print("✅ Все модели загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")

tokenized_text = tokenizer.texts_to_sequences([text])

max_len = model_keras.input_shape[1]
print(f"Максимальная длина последовательности: {max_len}")

padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')

pred = model_keras.predict(padded_text)

predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Вероятности по классам:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

### Второй датасет

In [ ]:

cfg = load_config("neural_network_second")

with mlflow.start_run(run_name='second_experiment_neural_network'):
    
    mlflow.set_tag('CNN', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art)
        print(f"✅ Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "Second_version.csv"
        ]
        
        for path in alternative_paths:
            try:
                art = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()

    tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
    tokenizer = Tokenizer(**tokenizer_config)
    
    tokenizer.fit_on_texts(df['span'])

    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state
    )

    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])

    X_train_pad = pad_sequences(X_train_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================

    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])

    # =====================================================================================================================================
    #                                         СОЗДАНИЕ МОДЕЛИ CNN
    # =====================================================================================================================================

    model = Sequential()

    # Embedding слой
    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim, 
        input_length=cfg.training.max_sequence_length
    ))

    # Conv1D слой
    model.add(Conv1D(
        filters=cfg.model.conv_filters,
        kernel_size=cfg.model.conv_kernel_size,
        activation=cfg.model.conv_activation
    ))

    model.add(Dropout(cfg.model.dropout_rate_1))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(cfg.model.dense_units_1, activation=cfg.model.dense_activation_1))

    model.add(Dropout(cfg.model.dropout_rate_2))

    model.add(Dense(cfg.model.dense_units_2, activation=cfg.model.dense_activation_2))

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer='adam', 
        metrics=['accuracy']
    )

    model.fit(
        X_train_pad, 
        y_train, 
        batch_size=cfg.training.batch_size, 
        epochs=cfg.training.epochs, 
        validation_data=(X_val_pad, y_val)
    )

    model.summary()

    test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred = model.predict(X_test_pad)
    pred_class = np.argmax(pred, axis=1)

    f1 = f1_score(pred_class, y_test, average='weighted')
    accuracy = accuracy_score(pred_class, y_test)

    print(f'f1-score у модели равен {f1:.4f}')
    print(f'accuracy у модели равен {accuracy:.4f}')

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('test_loss', test_loss)

    model.save(cfg.model.artifacts.model)

    joblib.dump(tokenizer, cfg.model.artifacts.tokenizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)

    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')

    mlflow.log_params({
        'num_words': cfg.tokenizer.num_words,
        'embedding_dim': cfg.model.embedding_dim,
        'conv_filters': cfg.model.conv_filters,
        'conv_kernel_size': cfg.model.conv_kernel_size,
        'dropout_rate_1': cfg.model.dropout_rate_1,
        'dropout_rate_2': cfg.model.dropout_rate_2,
        'dense_units_1': cfg.model.dense_units_1,
        'batch_size': cfg.training.batch_size,
        'epochs': cfg.training.epochs,
        'max_sequence_length': cfg.training.max_sequence_length
    })

    print("✅ CNN модель успешно обучена и залогирована!")

#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.tokenizer}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)
encoder = joblib.load(encoder_file)
tokenizer = joblib.load(tokenizer_file)

print("✅ Все модели загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================


text = cfg_inference.test_text

print("Выполняем предсказание...")


tokenized_text = tokenizer.texts_to_sequences([text])


max_len = model_keras.input_shape[1]
print(f"Максимальная длина последовательности: {max_len}")


padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')


pred = model_keras.predict(padded_text)


predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Вероятности по классам:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

### Третий датасет

In [ ]:
cfg = load_config("neural_network_third")

with mlflow.start_run(run_name='third_experiment_neural_network'):
    
    mlflow.set_tag('Bidirectional_GRU', cfg.model.version)
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art)
        print(f"✅ Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "third_experiment_dataset.csv"
        ]
        
        for path in alternative_paths:
            try:
                art = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df.head())

    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()
    mlflow.sklearn.autolog()

    # Создаем токенизатор с параметрами из конфига
    tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
    tokenizer = Tokenizer(**tokenizer_config)
    
    tokenizer.fit_on_texts(df['span'])

    sequences = tokenizer.texts_to_sequences(df['span'])

    # Автоматическое определение максимальной длины
    max_len = max(len(seq) for seq in sequences)
    print(f"📏 Максимальная длина последовательности: {max_len}")

    X = pad_sequences(sequences, maxlen=max_len)

    # =====================================================================================================================================
    #                                         ПРЕОБРАЗОВАНИЕ ТАРГЕТА
    # =====================================================================================================================================

    encoder = LabelEncoder()
    y = encoder.fit_transform(df['label'])
    num_classes = len(encoder.classes_)
    
    # =====================================================================================================================================
    #                                         РАЗДЕЛЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, 
        train_size=1-cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=y
    )

    X_test, X_val, y_test, y_val = train_test_split(
        X_temp, y_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=y_temp
    )

    print(f"📊 Размеры данных:")
    print(f"   Train: {X_train.shape}, {y_train.shape}")
    print(f"   Test: {X_test.shape}, {y_test.shape}")
    print(f"   Val: {X_val.shape}, {y_val.shape}")

    # =====================================================================================================================================
    #                                         СОЗДАНИЕ И ОБУЧЕНИЕ МОДЕЛИ
    # =====================================================================================================================================

    model = Sequential()

    # Embedding слой
    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim
    ))

    # Bidirectional GRU слой
    model.add(Bidirectional(GRU(cfg.model.gru_units)))

    # Dense слои
    model.add(Dense(cfg.model.dense_units, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer='adam', 
        metrics=['accuracy']
    )

    # Обучение
    model.fit(
        X_train, y_train, 
        epochs=cfg.training.epochs, 
        batch_size=cfg.training.batch_size, 
        validation_data=(X_val, y_val)
    )

    # =====================================================================================================================================
    #                                         ОЦЕНКА МОДЕЛИ
    # =====================================================================================================================================

    model.summary()

    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'📊 Test Loss: {loss:.4f}')
    print(f'📊 Test Accuracy: {accuracy:.4f}')

    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    f1 = f1_score(y_pred, y_test, average='weighted')
    acc_score = accuracy_score(y_pred, y_test)

    print(f'🎯 F1-score: {f1:.4f}')
    print(f'🎯 Accuracy: {acc_score:.4f}')

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МЕТРИК
    # =====================================================================================================================================

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', acc_score)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)

    # =====================================================================================================================================
    #                                         СОХРАНЕНИЕ АРТЕФАКТОВ
    # =====================================================================================================================================

    # Сохраняем модель
    model.save(cfg.model.artifacts.model)
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')

    # Сохраняем preprocessing объекты
    with open(cfg.model.artifacts.tokenizer, 'wb') as f:
        pickle.dump(tokenizer, f)
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'preprocessing')

    with open(cfg.model.artifacts.encoder, 'wb') as f:
        pickle.dump(encoder, f)
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'preprocessing')

    # Сохраняем информацию о предобработке
    preprocessing_info = {
        'vocab_size': len(tokenizer.word_index),
        'max_sequence_length': max_len,
        'num_classes': num_classes,
        'classes': list(encoder.classes_)
    }
    
    with open(cfg.model.artifacts.preprocessing_info, 'wb') as f:
        pickle.dump(preprocessing_info, f)
    mlflow.log_artifact(cfg.model.artifacts.preprocessing_info, 'preprocessing')

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ ПАРАМЕТРОВ
    # =====================================================================================================================================

    mlflow.log_params({
        "model_type": "Bidirectional_GRU",
        "embedding_dim": cfg.model.embedding_dim,
        "gru_units": cfg.model.gru_units,
        "dense_units": cfg.model.dense_units,
        "num_classes": num_classes,
        "vocab_size": len(tokenizer.word_index),
        "max_sequence_length": max_len,
        "batch_size": cfg.training.batch_size,
        "epochs": cfg.training.epochs
    })

    print("✅ Bidirectional GRU модель успешно обучена и залогирована!")

#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network_third")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'attributes.run_name = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

# Формируем пути к артефактам
model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"preprocessing/{cfg_inference.model.tokenizer}"
encoder_path = f"preprocessing/{cfg_inference.model.encoder}"
info_path = f"preprocessing/{cfg_inference.model.preprocessing_info}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    info_file = client.download_artifacts(latest_run_model_id, info_path)
    print("✅ Все модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)

with open(tokenizer_file, 'rb') as f:
    tokenizer = pickle.load(f)

with open(encoder_file, 'rb') as f:
    encoder = pickle.load(f)

with open(info_file, 'rb') as f:
    preprocessing_info = pickle.load(f)

print("✅ Все модели загружены в память")
print(f"📊 Информация о предобработке: {preprocessing_info}")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")

# Токенизация текста
tokenized_text = tokenizer.texts_to_sequences([text])
print(f"Токенизированный текст: {tokenized_text}")

# Получаем максимальную длину последовательности из preprocessing_info
max_len = preprocessing_info['max_sequence_length']
print(f"Максимальная длина последовательности: {max_len}")

# Паддинг
padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')
print(f"Текст после паддинга: {padded_text}")

# Предсказание
pred = model_keras.predict(padded_text)
print(f"Сырые предсказания: {pred}")

# Обработка результатов
predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)
confidence = np.max(pred, axis=1)[0]

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Уверенность: {confidence:.4f}')
print(f'Все вероятности:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

## Третий эксперимент (Трансформеры)

### Первый датасет

In [ ]:
cfg = load_config("transformers_first")

with mlflow.start_run(run_name='transformers_experiment_1'):
    
    mlflow.set_tag('Transformers', cfg.model.version)
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================
    

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")


    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art_loc = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art_loc)
        print(f"✅ Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")

        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "First_version.csv"
        ]
        
        for path in alternative_paths:
            try:
                art_loc = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art_loc)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")


    df = df[[cfg.data.text_column, cfg.data.label_column]]
    df = df.rename(columns={cfg.data.label_column: "labels"})
    display(df)

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ДАТАСЕТА/ТАРГЕТОВ
    # =====================================================================================================================================
    
    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['labels']
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['labels']
    )

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(df_train['labels'])
    y_test = encoder.transform(df_test['labels'])
    y_val = encoder.transform(df_val['labels'])

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА МОДЕЛИ
    # =====================================================================================================================================
    
    mlflow.transformers.autolog()


    tokenizer = AutoTokenizer.from_pretrained(cfg.model.model_name)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.model_name, 
        num_labels=len(encoder.classes_), 
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)}
    )
    

    dataset_train = Dataset.from_pandas(df_train.assign(labels=y_train))    
    dataset_test = Dataset.from_pandas(df_test.assign(labels=y_test))    
    dataset_val = Dataset.from_pandas(df_val.assign(labels=y_val))


    def tokenize_dataset(row):
        tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
        return tokenizer(row[cfg.data.text_column], **tokenizer_config)

    dataset_tokenized_train = dataset_train.map(tokenize_dataset, batched=False)
    dataset_tokenized_test = dataset_test.map(tokenize_dataset, batched=False)
    dataset_tokenized_val = dataset_val.map(tokenize_dataset, batched=False)


    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        return_tensors='pt'
    )
    

    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        overwrite_output_dir=True,
        logging_dir='./logs/',
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        eval_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        warmup_ratio=cfg.training.warmup_ratio,
        lr_scheduler_type=cfg.training.lr_scheduler_type,
        metric_for_best_model=cfg.training.metric_for_best_model,
        weight_decay=cfg.training.weight_decay,
        load_best_model_at_end=cfg.training.load_best_model_at_end,
        save_total_limit=cfg.training.save_total_limit,
        max_grad_norm=cfg.training.max_grad_norm,
        logging_steps=cfg.training.logging_steps
    )


    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        
        return {
            'f1-score': f1_score(labels, predictions, average='weighted'),
            'accuracy': accuracy_score(labels, predictions)
        }


    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=dataset_tokenized_train,
        eval_dataset=dataset_tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.training.early_stopping_patience)]
    )


    trainer.train()


    final_metrics = trainer.evaluate(dataset_tokenized_test)
    print(f"✅ Финальные метрики: {final_metrics}")


    # =====================================================================================================================================
    #                                         СОХРАНЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    # Получаем предсказания на тестовом наборе
    final_predictions = trainer.predict(dataset_tokenized_test)
    predictions = np.argmax(final_predictions.predictions, axis=1)
    labels = final_predictions.label_ids

    # Вычисляем нужные метрики
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    
    # Логируем метрики с нужными названиями
    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy) 
    
    # Также можно залогировать остальные метрики без префикса eval_
    for key, value in final_predictions.metrics.items():
        if key.startswith('eval_'):
            clean_key = key.replace('eval_', '')
            mlflow.log_metric(clean_key, value)

    print(f"✅ Final metrics - F1: {f1:.4f}, Accuracy: {accuracy:.4f}, Loss: {test_loss:.4f}")

    # Сохраняем артефакты
    model_dir = cfg.artifacts.model_dir
    tokenizer_dir = cfg.artifacts.tokenizer_dir

    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(tokenizer_dir, exist_ok=True)

    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)

    mlflow.log_artifacts(model_dir, "model")
    mlflow.log_artifacts(tokenizer_dir, "tokenizer")

    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)
    mlflow.log_artifact('label_encoder.pkl')

    mlflow.log_params({
        'model_name': cfg.model.model_name,
        'num_labels': len(encoder.classes_),
        'num_train_epochs': cfg.training.num_train_epochs,
        'learning_rate': cfg.training.learning_rate,
        'batch_size': cfg.training.per_device_train_batch_size,
        'early_stopping_patience': cfg.training.early_stopping_patience
    })

    print("✅ Трансформер успешно обучен и залогирован!")

#### Проверка результата

In [ ]:
cfg_inference = load_config("inference_transformers_first")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


print("Загружаем модель и токенизатор...")
try:

    model_dir = client.download_artifacts(latest_run_model_id, cfg_inference.model.artifacts_path)
    tokenizer_dir = client.download_artifacts(latest_run_model_id, cfg_inference.model.tokenizer_path)
    print("✅ Модель и токенизатор успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

print("✅ Модель и токенизатор загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================


text = cfg_inference.test_text

print("Выполняем предсказание...")


tokenizer_config = {k: v for k, v in cfg_inference.tokenizer.items() if v is not None}
inputs = tokenizer(text, **tokenizer_config)


with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

predicted_class_idx = predictions.argmax().item()
predicted_prob = predictions.max().item()

id2label = model.config.id2label
predicted_label = id2label[predicted_class_idx]

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_label}')
print(f'Вероятность: {predicted_prob:.4f}')
print(f'Все вероятности:')
for i, prob in enumerate(predictions[0]):
    label = id2label[i]
    print(f'  {label}: {prob:.4f}')
print('=' * 100)

### Второй датасет

In [ ]:
cfg = load_config("transformers_second")

with mlflow.start_run(run_name='transformers_experiment_2'):
    
    mlflow.set_tag('Transformers', cfg.model.version)
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art_loc = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art_loc)
        print(f"✅ Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")

        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "second_experiment_dataset.csv"
        ]
        
        for path in alternative_paths:
            try:
                art_loc = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art_loc)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df.head())

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ДАННЫХ
    # =====================================================================================================================================

    encoder = LabelEncoder()
    df['label_encoded'] = encoder.fit_transform(df[cfg.data.label_column])
    df = df.rename(columns={'label_encoded': 'labels'})
    df = df[[cfg.data.text_column, 'labels']]

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА МОДЕЛИ
    # =====================================================================================================================================

    # ВЫКЛЮЧАЕМ autolog - будем логировать вручную для контроля
    # mlflow.transformers.autolog()

    tokenizer = AutoTokenizer.from_pretrained(cfg.model.model_name)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.model_name,
        num_labels=len(encoder.classes_),
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)}
    )
    
    # =====================================================================================================================================
    #                                         РАЗДЕЛЕНИЕ ДАННЫХ
    # =====================================================================================================================================

    df_train, df_temp = train_test_split(
        df, 
        train_size=1-cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['labels']
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['labels']
    )

    # Создаем datasets
    dataset_train = Dataset.from_pandas(df_train)
    dataset_test = Dataset.from_pandas(df_test)
    dataset_val = Dataset.from_pandas(df_val)

    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ
    # =====================================================================================================================================

    def tokenize_dataset(batch):
        tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
        return tokenizer(
            batch[cfg.data.text_column],
            **tokenizer_config
        )

    tokenized_train = dataset_train.map(tokenize_dataset, batched=True)
    tokenized_test = dataset_test.map(tokenize_dataset, batched=True)
    tokenized_val = dataset_val.map(tokenize_dataset, batched=True)

    # =====================================================================================================================================
    #                                         DATA COLLATOR
    # =====================================================================================================================================

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        max_length=cfg.tokenizer.max_length,
        return_tensors="pt"
    )

    # =====================================================================================================================================
    #                                         TRAINING ARGUMENTS
    # =====================================================================================================================================

    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        eval_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        logging_dir=cfg.training.logging_dir,
        logging_steps=cfg.training.logging_steps,
        load_best_model_at_end=cfg.training.load_best_model_at_end,
        metric_for_best_model=cfg.training.metric_for_best_model
    )

    # =====================================================================================================================================
    #                                         МЕТРИКИ
    # =====================================================================================================================================

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        y_pred = np.argmax(predictions, axis=1)

        return {
            'f1_score': f1_score(y_pred, labels, average='weighted'),
            'accuracy': accuracy_score(y_pred, labels)
        }
    
    # =====================================================================================================================================
    #                                         TRAINER И ОБУЧЕНИЕ
    # =====================================================================================================================================

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    # Обучение
    trainer.train()

    # =====================================================================================================================================
    #                                         ОЦЕНКА И ЛОГИРОВАНИЕ МЕТРИК
    # =====================================================================================================================================

    # Получаем предсказания для вычисления метрик
    test_predictions = trainer.predict(tokenized_test)
    test_preds = np.argmax(test_predictions.predictions, axis=1)
    test_labels = test_predictions.label_ids

    # ВЫЧИСЛЯЕМ МЕТРИКИ ВРУЧНУЮ с нужными названиями
    accuracy = accuracy_score(test_labels, test_preds)
    f1_score_value = f1_score(test_labels, test_preds, average='weighted')

    # ЛОГИРУЕМ МЕТРИКИ с точными названиями
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1_score_value)

    print(f"🎯 Финальные метрики:")
    print(f"   accuracy: {accuracy:.4f}")
    print(f"   f1_score: {f1_score_value:.4f}")

    # =====================================================================================================================================
    #                                         СОХРАНЕНИЕ АРТЕФАКТОВ
    # =====================================================================================================================================

    # Создаем временные папки для артефактов
    model_dir = cfg.artifacts.model_dir
    tokenizer_dir = cfg.artifacts.tokenizer_dir
    
    # Очищаем папки если существуют
    import shutil
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
    if os.path.exists(tokenizer_dir):
        shutil.rmtree(tokenizer_dir)

    # Сохраняем модель и токенизатор
    trainer.save_model(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)

    # Сохраняем encoder
    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)

    # ЛОГИРУЕМ АРТЕФАКТЫ в нужной структуре
    mlflow.log_artifacts(model_dir, "model")
    mlflow.log_artifacts(tokenizer_dir, "tokenizer")
    mlflow.log_artifact('label_encoder.pkl')

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ ПАРАМЕТРОВ
    # =====================================================================================================================================

    mlflow.log_params({
        'model_name': cfg.model.model_name,
        'num_labels': len(encoder.classes_),
        'num_train_epochs': cfg.training.num_train_epochs,
        'learning_rate': cfg.training.learning_rate,
        'batch_size': cfg.training.per_device_train_batch_size,
        'test_size': cfg.training.test_size,
        'val_size': cfg.training.val_size
    })

    # =====================================================================================================================================
    #                                         ОЧИСТКА ВРЕМЕННЫХ ФАЙЛОВ
    # =====================================================================================================================================

    shutil.rmtree(model_dir)
    shutil.rmtree(tokenizer_dir)
    os.remove('label_encoder.pkl')

    print("✅ Трансформер успешно обучен и залогирован!")

#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_transformers_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'attributes.run_name = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

print("Загружаем модель, токенизатор и энкодер...")
try:
    model_dir = client.download_artifacts(latest_run_model_id, 'model')
    tokenizer_dir = client.download_artifacts(latest_run_model_id, 'tokenizer')
    encoder_path = client.download_artifacts(latest_run_model_id, 'label_encoder.pkl')
    print("✅ Модель, токенизатор и энкодер успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

# Загружаем encoder
with open(encoder_path, 'rb') as f:
    encoder = pickle.load(f)

print("✅ Модель, токенизатор и энкодер загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")

# Конфиг токенизатора из конфига
tokenizer_config = {k: v for k, v in cfg_inference.tokenizer.items() if v is not None}
inputs = tokenizer(text, **tokenizer_config)

# Предсказание
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

predicted_class_idx = predictions.argmax().item()
predicted_prob = predictions.max().item()

# Используем encoder для получения оригинального названия класса
predicted_label = encoder.inverse_transform([predicted_class_idx])[0]

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_label}')
print(f'Вероятность: {predicted_prob:.4f}')
print(f'Все вероятности:')
for i, prob in enumerate(predictions[0]):
    label = encoder.inverse_transform([i])[0]
    print(f'  {label}: {prob:.4f}')
print('=' * 100)

### Третий датасет

In [ ]:
cfg = load_config("transformers_third")

with mlflow.start_run(run_name=cfg.mlflow.run_name):

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)
    mlflow.log_param("model_name", cfg.model.name)

    # ---------------------------
    #   Загружаем датасет
    # ---------------------------

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_name], 
        filter_string=f"tags.mlflow.runName = 'Third dataset'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError("Не найден run с датасетом 'Third dataset'")

    dataset_run_id = dataset_runs[0].info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    dataset_path_in_run = "datasets/third_experiment_dataset.csv" 
    artifact_local_path = client.download_artifacts(dataset_run_id, dataset_path_in_run)

    df = pd.read_csv(artifact_local_path)
    print(f"✅ Датасет загружен из run: {artifact_local_path}")

    # ---------------------------
    #   ПОДГОТОВКА ДАННЫХ
    # ---------------------------

    # Кодируем лейблы
    encoder = LabelEncoder()
    df["labels"] = encoder.fit_transform(df["label"])
    df["labels"] = df["labels"].astype(int)

    mlflow.log_param("num_labels", len(encoder.classes_))

    # Разделяем на train/val/test
    df_train, df_temp = train_test_split(
        df,
        test_size=1 - cfg.data.train_size,
        random_state=cfg.data.random_state,
        shuffle=True,
        stratify=df["labels"]
    )

    df_val, df_test = train_test_split(
        df_temp,
        test_size=cfg.data.val_size,
        random_state=cfg.data.random_state,
        shuffle=True,
        stratify=df_temp["labels"]
    )

    print(f"📊 Размеры данных: Train={len(df_train)}, Val={len(df_val)}, Test={len(df_test)}")

    # ---------------------------
    #   ПОДГОТОВКА ДАТАСЕТОВ (ИСПРАВЛЕННАЯ)
    # ---------------------------

    # Сбрасываем индексы чтобы избежать __index_level_0__
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    # Создаем датасеты
    dataset_train = Dataset.from_dict({
        "text": df_train["span"].tolist(),
        "labels": df_train["labels"].tolist()
    })
    
    dataset_val = Dataset.from_dict({
        "text": df_val["span"].tolist(), 
        "labels": df_val["labels"].tolist()
    })
    
    dataset_test = Dataset.from_dict({
        "text": df_test["span"].tolist(),
        "labels": df_test["labels"].tolist()
    })

    # ---------------------------
    #   ТОКЕНИЗАЦИЯ
    # ---------------------------

    tokenizer = AutoTokenizer.from_pretrained(cfg.model.name)

    def tokenize_function(examples):
        # Токенизируем только текст
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            padding=False,
            max_length=cfg.model.max_length
        )
        return tokenized

    # Применяем токенизацию
    tokenized_train = dataset_train.map(tokenize_function, batched=True)
    tokenized_val = dataset_val.map(tokenize_function, batched=True)
    tokenized_test = dataset_test.map(tokenize_function, batched=True)

    # Удаляем текстовую колонку после токенизации
    tokenized_train = tokenized_train.remove_columns(['text'])
    tokenized_val = tokenized_val.remove_columns(['text'])
    tokenized_test = tokenized_test.remove_columns(['text'])

    print("🔍 Проверка структуры данных после токенизации:")
    print(f"Train columns: {tokenized_train.column_names}")
    print(f"Sample train: {tokenized_train[0]}")

    # DataCollator для динамического паддинга
    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        max_length=cfg.model.max_length,
        return_tensors="pt"
    )

    # ---------------------------
    #   ИНИЦИАЛИЗАЦИЯ МОДЕЛИ
    # ---------------------------

    num_labels = len(encoder.classes_)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.name,
        num_labels=num_labels,
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)},
        hidden_dropout_prob=cfg.model.dropout.hidden,
        attention_probs_dropout_prob=cfg.model.dropout.attention,
        classifier_dropout=cfg.model.dropout.classifier
    )

    # ---------------------------
    #   TRAINING ARGUMENTS
    # ---------------------------

    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        warmup_ratio=cfg.training.warmup_ratio,
        lr_scheduler_type=cfg.training.lr_scheduler_type,
        eval_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        weight_decay=cfg.training.weight_decay,
        logging_steps=cfg.training.logging_steps,
        save_total_limit=cfg.training.save_total_limit,
        max_grad_norm=cfg.training.max_grad_norm,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        dataloader_pin_memory=False,
        dataloader_num_workers=0,
        remove_unused_columns=True  # Включаем удаление лишних колонок
    )

    # ---------------------------
    #   МЕТРИКИ
    # ---------------------------

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=1)

        return {
            "f1": f1_score(labels, preds, average="weighted"),
            "accuracy": accuracy_score(labels, preds)
        }

    # ---------------------------
    #   ОБУЧЕНИЕ
    # ---------------------------

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        # tokenizer=tokenizer,  # УБИРАЕМ эту строку (deprecated)
        callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.training.early_stopping_patience)]
    )

    print("🚀 Начинаем обучение...")
    trainer.train()

    # ---------------------------
    #   ОЦЕНКА И ЛОГИРОВАНИЕ
    # ---------------------------
    
    print("📊 Оцениваем модель на тестовом наборе...")
    test_predictions = trainer.predict(tokenized_test)
    test_preds = np.argmax(test_predictions.predictions, axis=1)
    test_labels = test_predictions.label_ids
    
    # Вычисляем метрики
    test_f1 = f1_score(test_labels, test_preds, average="weighted")
    test_accuracy = accuracy_score(test_labels, test_preds)
    
    # Логируем метрики
    mlflow.log_metric("f1_score", test_f1)
    mlflow.log_metric("accuracy", test_accuracy)
    
    print(f"🎯 Финальные метрики на тесте:")
    print(f"   F1-score: {test_f1:.4f}")
    print(f"   Accuracy: {test_accuracy:.4f}")

    # ---------------------------
    #   СОХРАНЕНИЕ
    # ---------------------------

    # Создаем временные папки с правильными путями
    model_dir = "best_transformer_model"
    tokenizer_dir = "best_transformer_tokenizer"

    # Сохраняем модель и токенизатор
    trainer.save_model(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)

    # Логируем папки как артефакты (ИСПРАВЛЕНО)
    mlflow.log_artifacts(model_dir, artifact_path="model")    # ✅ log_artifacts для папки
    mlflow.log_artifacts(tokenizer_dir, artifact_path="tokenizer")

    # Сохраняем encoder
    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)
    mlflow.log_artifact('label_encoder.pkl', artifact_path="preprocessing")

    # Очищаем временные файлы (опционально)
    import shutil
    shutil.rmtree(model_dir)
    shutil.rmtree(tokenizer_dir)
    os.remove('label_encoder.pkl')

    print("✅ Трансформер успешно обучен и залогирован!")

#### Проверка результатов (инференс)

In [ ]:
cfg = load_config("inference_transformers_third")

client = MlflowClient(tracking_uri=cfg.mlflow.tracking_uri)

# ================================
# 2. Ищем нужный run в MLflow
# ================================
print("Ищем run с моделью...")

runs = client.search_runs(
    experiment_ids=[cfg.mlflow.experiment_id],
    filter_string=f'attributes.run_name = "{cfg.model.run_name}"',  # ⚠️ исправлено на attributes.run_name
    order_by=["attributes.end_time desc"]
)

if not runs:
    raise ValueError(f"❌ Run '{cfg.model.run_name}' не найден!")

run_id = runs[0].info.run_id
print(f"✅ Найден run: {run_id}")

# ================================
# 3. Скачиваем артефакты
# ================================
print("Скачиваем модель, токенизатор и энкодер...")

try:
    model_dir = client.download_artifacts(run_id, cfg.model.artifacts_path)
    tokenizer_dir = client.download_artifacts(run_id, cfg.model.tokenizer_path)
    encoder_path = client.download_artifacts(run_id, "preprocessing/label_encoder.pkl")
    
    print(f"📁 Модель скачана: {model_dir}")
    print(f"📁 Токенизатор скачан: {tokenizer_dir}")
    print(f"📁 Энкодер скачан: {encoder_path}")
    
except Exception as e:
    print(f"❌ Ошибка загрузки артефактов: {e}")
    raise

# ================================
# 4. Загружаем модель в память
# ================================
print("Загружаем модель и компоненты...")

model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

# Загружаем encoder
with open(encoder_path, 'rb') as f:
    encoder = pickle.load(f)

print("✅ Все компоненты загружены")

# ================================
# 5. Предсказание
# ================================
text = cfg.test_text
print(f"Тестовый текст: «{text}»")

# Конфиг токенизатора
tokenizer_cfg = {k: v for k, v in cfg.tokenizer.items() if v is not None}

# Токенизация
inputs = tokenizer(text, **tokenizer_cfg)

# Предсказание
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Обработка результатов
pred_idx = probs.argmax().item()
pred_prob = probs[0][pred_idx].item()

# Используем наш encoder для получения оригинального названия класса
pred_label = encoder.inverse_transform([pred_idx])[0]

print("\n" + "="*100)
print(f"Исходный текст: «{text}»")
print(f"Предсказанная метка: {pred_label}")
print(f"Вероятность: {pred_prob:.4f}")
print("\nВсе вероятности:")
for i, p in enumerate(probs[0]):
    label = encoder.inverse_transform([i])[0]
    print(f"  {label}: {p:.4f}")
print("="*100)

# Выбор лучшей модели (для инференса)

In [ ]:
client = MlflowClient()

runs = client.search_runs(experiment_ids = ['0'],
                          order_by = ['metrics.f1_score desc', 'metrics.accuracy desc'])

print(runs[0])